# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
reaction_kcat_file = "./data/reaction_kappori.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01ac5a207940
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [22]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

# Step4: Calibration parameters

Compare with C13 data

In [23]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])


0.4191429334139575


In [24]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ICL', 'PTAr', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'ACKr_reverse', 'SUCDi']


Calibration kcat according Enzyme usage 

In [27]:
len(c13reaction)

38

In [30]:
round_num = 1
f = 0.406
ptot = 0.56 
sigma = 1
enz_total = round(ptot * f * sigma, 3)
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
reaction_flux_file = './analysis/ECMpy_kapp_adj_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,reaction_flux_file)
print('model original biomass :')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)
enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

save_file="./analysis/reaction_kapp_change_c13_enz_usage.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)
reaction_flux_file = './analysis/ECMpy_kapp_adj_solution_df_pfba.csv'
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13_enz_usage.csv" 
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_reaction_enz_usage_df.csv'
model_file = './data/iML1515.xml' 
json_model_file="./model/iML1515_irr_enz_constraint_adj.json"
lowerbound = 0 

change_reaction_list =[]

while(len(c13reaction_2_enz_model_diff)>2 and len(c13reaction)>round_num):
    print(round_num)

    for eachreaction in c13reaction:
        if eachreaction in change_reaction_list:
            pass
        else:
            print(eachreaction)
            select_reaction = [eachreaction]
            change_reaction_list.append(eachreaction)
            break

    change_reaction_kcat_by_autopacmen(select_reaction,reaction_kcat_mw_file,reaction_kcat_mw_file)
    trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, enz_total, json_model_file)

    enz_model=get_enzyme_constraint_model(json_model_file)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,reaction_flux_file)
    print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

    enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
    enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
    c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
    print('C13 reaction diff :')
    print (c13reaction_2_enz_model_diff)  
    round_num = round_num+1
    print('changed reaction list :')
    print(change_reaction_list)

model original biomass :
0.4191429334139575
['ICL', 'PTAr', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'ACKr_reverse', 'SUCDi']
1
PGI
0.4247972025156424
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['PGI']
2
PFK
0.4247972025156424
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['PGI', 'PFK']
3
FBA
0.4398888585784848
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['PGI', 'PFK', 'FBA']
4
TPI
0.44287829290112285
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['PGI', 'PFK', 'FBA', 'TPI']
5
GAPD
0.47834528435815915
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['PGI', 'PFK', 'FBA', 'TPI', 'GAPD']
6
PGK_reverse
0.4861574416120893
C13 reaction diff :
['ICL', 'AKGDH', 'MALS',

In [26]:
round_num = 1
f = 0.406
ptot = 0.56 
sigma = 1
enz_total = round(ptot * f * sigma, 3)
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
reaction_flux_file = './analysis/ECMpy_kapp_adj_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,reaction_flux_file)
print('model original biomass :')
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)
enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

save_file="./analysis/reaction_kapp_change_c13_enz_usage.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)
reaction_flux_file = './analysis/ECMpy_kapp_adj_solution_df_pfba.csv'
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13_enz_usage.csv" 
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_reaction_enz_usage_df.csv'
model_file = './data/iML1515.xml' 
json_model_file="./model/iML1515_irr_enz_constraint_adj.json"
lowerbound = 0 

change_reaction_list =[]

while(len(c13reaction_2_enz_model_diff)>2 and len(reaction_kcat_mw.index)>round_num):
    print(round_num)
    reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
    print(reaction_enz_usage_df.head(5))

    for eachreaction in reaction_enz_usage_df.index:
        if eachreaction in change_reaction_list:
            pass
        else:
            print(eachreaction)
            select_reaction = [eachreaction]
            change_reaction_list.append(eachreaction)
            break

    change_reaction_kcat_by_autopacmen(select_reaction,reaction_kcat_mw_file,reaction_kcat_mw_file)
    trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, enz_total, json_model_file)

    enz_model=get_enzyme_constraint_model(json_model_file)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,reaction_flux_file)
    print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

    enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
    enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
    c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
    print('C13 reaction diff :')
    print (c13reaction_2_enz_model_diff)  
    round_num = round_num+1
    print('changed reaction list :')
    print(change_reaction_list)

model original biomass :
0.4191429334139575
['ICL', 'PTAr', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'ACKr_reverse', 'SUCDi']
1
              kcat_mw       flux  enz useage  enz ratio
GAPD       734.538803  17.921312    0.024398   0.107480
ENO        946.228175  17.224202    0.018203   0.080189
FBA        478.270272   8.565138    0.017909   0.078892
NADH16pp  1295.628496  17.522567    0.013524   0.059579
RBFSa        0.037137   0.000374    0.010068   0.044350
GAPD
0.45504524963917803
C13 reaction diff :
['ICL', 'AKGDH', 'MALS', 'PDH', 'ME2', 'SUCOAS_reverse', 'SUCDi']
changed reaction list :
['GAPD']
2
              kcat_mw       flux  enz useage  enz ratio
ENO        946.228175  17.212136    0.018190   0.080133
FBA        478.270272   8.667931    0.018124   0.079839
PFL        580.563567   8.720638    0.015021   0.066172
NADH16pp  1295.628496  17.536057    0.013535   0.059625
RBFSa        0.037137   0.000406    0.010930   0.048149
ENO
0.47950285543969334
C13 reaction diff :
['

SystemError: <built-in function delete_intPtr> returned a result with an error set

In [43]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.01].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"

change_reaction_list=change_reaction_kcat_by_autopacmen(select_reaction,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list)

['PDH', 'GAPD', 'NADH16pp', 'ENO', 'RBFSa', 'KARA2', 'KARA1_reverse', 'FBA', 'METS', 'GHMT2r', 'GLNS', 'GND', 'PGK_reverse', 'PGI', 'PPC', 'METAT', 'ATPS4rpp', 'ARGSS', 'GTPCI', 'PGM_reverse', 'MOHMT']
['PDH', 'GAPD', 'NADH16pp', 'ENO', 'RBFSa', 'KARA2', 'KARA1_reverse', 'FBA', 'METS', 'GHMT2r', 'GLNS', 'GND', 'PGK_reverse', 'PGI', 'PPC', 'METAT', 'GTPCI', 'PGM_reverse', 'MOHMT']


In [44]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [45]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.7041020384688522


In [46]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ME2', 'PFL', 'SUCOAS_reverse']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698